In [211]:
from inspect import signature

### 2.3 plain parsing
try this https://www.imdb.com/find?q=wonder+woman

In [4]:
import requests
url = 'https://www.imdb.com/find?q=wonder+woman'
r = requests.get(url)
text = r.text

In [5]:
print(text.replace('\n','      ')[:3000])

                  <!DOCTYPE html>      <html          xmlns:og="http://ogp.me/ns#"          xmlns:fb="http://www.facebook.com/2008/fbml">          <head>                                   <meta charset="utf-8">              <meta http-equiv="X-UA-Compatible" content="IE=edge">                                      <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>            <script>          if (typeof uet == 'function') {            uet("bb", "LoadTitle", {wb: 1});          }      </script>        <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>              <title>Find - IMDb</title>        <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>      <script>          if (typeof uet == 'function') {            uet("be", "LoadTitle", {wb: 1});          }      </script>      <script>          if (typeof u

In [6]:
find = text.find('Wonder')-100
text[find - 1000:find+1000]

'       <!-- no content received for slot: injected_billboard -->\n\t\n<div id="content-2-wide">\n    <div id="main">\n        <div class="article">\n       <h1 class="findHeader">Results for <span class="findSearchTerm">"wonder woman"</span></h1>\n\n\n        <div id="findSubHeader"><span id="findSubHeaderLabel">Jump to:</span>\n              <a href="#tt">Titles</a> \n              <span class="ghost">|</span>\n              <a href="#nm">Names</a> \n              <span class="ghost">|</span>\n              <a href="#kw">Keywords</a> \n              <span class="ghost">|</span>\n              <a href="#co">Companies</a> \n              \n        </div>\n\n<div class="findSection">\n<h3 class="findSectionHeader"><a name="tt"></a>Titles</h3>\n<table class="findList">\n<tr class="findResult odd"> <td class="primary_photo"> <a href="/title/tt0451279/" ><img src="https://m.media-amazon.com/images/M/MV5BMTYzODQzYjQtNTczNC00MzZhLTg1ZWYtZDUxYmQ3ZTY4NzA1XkEyXkFqcGdeQXVyODE5NzE3OTE@._V1_UX32_C

## Classic sequential parsing

In [70]:
import time
import sys
#time?

In [71]:
url = 'https://www.imdb.com/find?q='
names = ['wonder woman', 'batman', 'tenet']*10
texts = []

In [74]:
def get_classic_text(url):
    with requests.Session() as session:
        with session.get(url) as response:
            text = response.text
            sys.stdout.write('.')
            texts.append(text)

def run_classic(url):
    tasks = (get_classic_text(url+i) for i in names)
    print(str(len(names))+' tasks appended, running them now')
    list(tasks)

In [75]:
%time run_classic(url)

30 tasks appended, running them now
..............................CPU times: user 658 ms, sys: 93 ms, total: 751 ms
Wall time: 19.1 s


In [76]:
texts[0][:100]

'\n\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008'

## Modern asynchronous

In [86]:
import asyncio
import aiohttp

names = ['wonder woman', 'batman', 'tenet']*100

texts = []

In [87]:
async def get_async_text(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            text =  await response.text()
            sys.stdout.write('.')
            texts.append(text)

In [88]:
async def run_async(url):
    tasks = [asyncio.ensure_future(get_async_text(url+i)) for i in names]        
    print(str(len(names))+' tasks appended, running them now')
    await asyncio.wait(tasks)

In [89]:
t1=time.time()
await run_async(url);
t2=time.time()
print('\n', round(t2-t1), 'seconds passed')

300 tasks appended, running them now
............................................................................................................................................................................................................................................................................................................
 8 seconds passed


In [90]:
texts[0][:100]

'\n\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008'

### Parsing per se
there are two packages
- bs4 slow but intuitive
- lxml.html fast but bare bones

In [102]:
text = texts[0]

In [103]:
from IPython.core.display import display, HTML
display(HTML(text))

### Lets try the soup
- find_all()
- find()
- get()
- text

In [142]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(text)

In [143]:
results = soup.find_all('td')

In [159]:
results

[<td class="primary_photo"> <a href="/title/tt6723592/"><img src="https://m.media-amazon.com/images/M/MV5BYzg0NGM2NjAtNmIxOC00MDJmLTg5ZmYtYzM0MTE4NWE2NzlhXkEyXkFqcGdeQXVyMTA4NjE0NjEy._V1_UX32_CR0,0,32,44_AL_.jpg"/></a> </td>,
 <td class="result_text"> <a href="/title/tt6723592/">Довод</a> (2020) <br/>aka <i>"Tenet"</i> </td>,
 <td class="primary_photo"> <a href="/title/tt13376362/"><img src="https://m.media-amazon.com/images/M/MV5BMjk4Njc3YzctMjEzOC00Y2Q0LWI2MTktZDE1M2RlMWRjZDVhXkEyXkFqcGdeQXVyMjY4MTU3NzQ@._V1_UX32_CR0,0,32,44_AL_.jpg"/></a> </td>,
 <td class="result_text"> <a href="/title/tt13376362/">Tenet</a> (2020) (TV Episode) <br/> <small>- Season 5 <span class="ghost">|</span> Episode 22 </small> <br/><small>- <a href="/title/tt13338556/">CritiCar</a> (2016) (TV Series) </small> </td>,
 <td class="primary_photo"> <a href="/title/tt15035860/"><img src="https://m.media-amazon.com/images/M/MV5BYTVlYzMwYmItMmI5Yi00M2IxLWI2YzUtMTA0ZTY4NGFjMjkyXkEyXkFqcGdeQXVyNDI0NzUxMTI@._V1_UY44_CR2

In [154]:
iresults = iter(results)

In [155]:
next(iresults)

<td class="primary_photo"> <a href="/title/tt6723592/"><img src="https://m.media-amazon.com/images/M/MV5BYzg0NGM2NjAtNmIxOC00MDJmLTg5ZmYtYzM0MTE4NWE2NzlhXkEyXkFqcGdeQXVyMTA4NjE0NjEy._V1_UX32_CR0,0,32,44_AL_.jpg"/></a> </td>

In [156]:
next(iresults)

<td class="result_text"> <a href="/title/tt6723592/">Довод</a> (2020) <br/>aka <i>"Tenet"</i> </td>

In [161]:
href = results[1].find('a').get('href')
href

'/title/tt6723592/'

In [158]:
results[1].find('a').text

'Довод'

In [164]:
text = requests.get('https://www.imdb.com/'+href).text

In [165]:
display(HTML(text))

In [173]:
soup = BeautifulSoup(text)

In [ ]:
results = soup.find_all('script')

In [206]:
find = text.find('Nolan')

In [208]:
text[find-200:find+2000]

'wb: 1}); }</script><script>if(typeof uet === \'function\'){ uet(\'bb\', \'LoadTitle\', {wb: 1}); }</script><title>Довод (2020) - IMDb</title><meta name="description" content="Довод: Directed by Christopher Nolan. With Juhan Ulfsak, Jefferson Hall, Ivo Uukkivi, Andrew Howard. Armed with only one word, Tenet, and fighting for the survival of the entire world, a Protagonist journeys through a twilight world of international espionage on a mission that will unfold in something beyond real time." data-id="main"/><script type="application/ld+json">{"@context":"https://schema.org","@type":"Movie","url":"/title/tt6723592/","name":"Tenet","alternateName":"Довод","image":"https://m.media-amazon.com/images/M/MV5BYzg0NGM2NjAtNmIxOC00MDJmLTg5ZmYtYzM0MTE4NWE2NzlhXkEyXkFqcGdeQXVyMTA4NjE0NjEy._V1_.jpg","description":"Armed with only one word, Tenet, and fighting for the survival of the entire world, a Protagonist journeys through a twilight world of international espionage on a mission that will unfo

In [212]:
signature(soup.find_all)

<Signature (name=None, attrs={}, recursive=True, text=None, limit=None, **kwargs)>

In [231]:
myjson = soup.find('script', attrs={"type":"application/ld+json"}).string

In [247]:
import json
from pprint import pprint
myjson = eval(myjson)

In [248]:
pprint(myjson)

{'@context': 'https://schema.org',
 '@type': 'Movie',
 'actor': [{'@type': 'Person',
            'name': 'John David Washington',
            'url': '/name/nm0913475/'},
           {'@type': 'Person',
            'name': 'Robert Pattinson',
            'url': '/name/nm1500155/'},
           {'@type': 'Person',
            'name': 'Elizabeth Debicki',
            'url': '/name/nm4456120/'}],
 'aggregateRating': {'@type': 'AggregateRating',
                     'bestRating': 10,
                     'ratingCount': 416421,
                     'ratingValue': 7.4,
                     'worstRating': 1},
 'alternateName': 'Довод',
 'contentRating': '16+',
 'creator': [{'@type': 'Organization', 'url': '/company/co0002663/'},
             {'@type': 'Organization', 'url': '/company/co0147954/'},
             {'@type': 'Organization', 'url': '/company/co0002663/'},
             {'@type': 'Person',
              'name': 'Christopher Nolan',
              'url': '/name/nm0634240/'}],
 'datePublis